<a href="https://colab.research.google.com/github/kskola/sql_project/blob/main/sales_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
# Install Required Libraries
!pip install sqlalchemy pandas

In [41]:
import pandas as pd
from sqlalchemy import create_engine, text

# Load the dataset into a DataFrame
df = pd.read_csv('/content/superstore.csv')

# Check the column names
print("Column names in the DataFrame:")
print(df.columns)

Column names in the DataFrame:
Index(['Customer.ID', 'Product.ID', 'Product.Name', 'Category', 'State',
       'City', 'Country', 'Discount', 'Market', 'Order.Date', 'Profit',
       'Quantity', 'Row.ID', 'Sales', 'Segment', 'Ship.Date', 'Ship.Mode',
       'Shipping.Cost', 'Sub.Category', 'Year', 'weeknum'],
      dtype='object')


In [44]:
# Calculate total_sales before inserting into the database
# Adjust the column names based on the output from the print statement
if 'Quantity' in df.columns and 'Sales' in df.columns:
    df['total_sales'] = df['Quantity'] * df['Sales']
else:
    print("One or both of the columns 'Quantity' and 'Sales' do not exist in the DataFrame.")

In [45]:
# Create a SQLite database in memory
engine = create_engine('sqlite:///:memory:')

# Create the table with the specified schema
create_table_query = """
CREATE TABLE superstore (
    "Customer.ID" TEXT,
    "Product.ID" TEXT,
    "Product.Name" TEXT,
    Category TEXT,
    State TEXT,
    City TEXT,
    Country TEXT,
    Discount NUMERIC,
    Market TEXT,
    "Order.Date" TEXT,
    Profit NUMERIC,
    Quantity NUMERIC,
    "Row.ID" NUMERIC,
    sales NUMERIC,
    segment TEXT,
    "Ship.Date" TEXT,
    "Ship.Mode" TEXT,
    "Shipping.Cost" NUMERIC,
    "Sub.Category" TEXT,
    Year NUMERIC,
    weeknum NUMERIC,
    total_sales NUMERIC
);
"""

# Execute the create table query
with engine.connect() as connection:
    connection.execute(text(create_table_query))

In [46]:
# Insert the DataFrame into the SQL table
df.to_sql('superstore', con=engine, if_exists='append', index=False)

# Step 4: Fetch and display the first 5 rows from the superstore table
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM superstore LIMIT 5;"))
    df_results = pd.DataFrame(result.fetchall(), columns=result.keys())

# Display the results in a table format
df_results

,Customer.ID,Product.ID,Product.Name,Category,State,City,Country,Discount,Market,Order.Date,...,Row.ID,sales,segment,Ship.Date,Ship.Mode,Shipping.Cost,Sub.Category,Year,weeknum,total_sales
0,LS-172304,OFF-PA-10002005,Xerox 225,Office Supplies,California,Los Angeles,United States,0,US,07/01/2011,...,36624,19,Consumer,09/01/2011,Second Class,4.37,Paper,2011,2,57
1,MV-174854,OFF-PA-10002893,"Wirebound Service Call Books, 5 1/2"" x 4""",Office Supplies,California,Los Angeles,United States,0,US,21/01/2011 00:00,...,37033,19,Consumer,26/01/2011,Standard Class,0.94,Paper,2011,4,38
2,CS-121304,OFF-PA-10000659,"Adams Phone Message Book, Professional, 400 Me...",Office Supplies,California,Los Angeles,United States,0,US,05/08/2011,...,31468,21,Consumer,09/08/2011,Standard Class,1.81,Paper,2011,32,63
3,CS-121304,OFF-PA-10001144,Xerox 1913,Office Supplies,California,Los Angeles,United States,0,US,05/08/2011,...,31469,111,Consumer,09/08/2011,Standard Class,4.59,Paper,2011,32,222
4,AP-109154,OFF-PA-10002105,Xerox 223,Office Supplies,California,Los Angeles,United States,0,US,29/09/2011,...,32440,6,Consumer,03/10/2011,Standard Class,1.32,Paper,2011,40,6


In [49]:
# Analysis of impact of discounts on sales and profits
query = """
SELECT
    "Product.Name",
    Category,
    SUM(total_sales) AS total_sales,
    SUM(Profit) AS total_profit,
    SUM(Discount) AS total_discount
FROM
    superstore
WHERE
    Discount > 0  -- Only consider orders with discounts
GROUP BY
    "Product.Name",
    Category
ORDER BY
    total_profit DESC
LIMIT 10;
"""

# Execute the query and fetch results into a DataFrame
with engine.connect() as connection:
    result = connection.execute(text(query))
    df_discount_analysis = pd.DataFrame(result.fetchall(), columns=result.keys())

df_discount_analysis

,Product.Name,Category,total_sales,total_profit,total_discount
0,Canon imageCLASS 2200 Advanced Copier,Technology,78400,5039.98560,0.600
1,"SAFCO Executive Leather Armchair, Black",Furniture,143979,3988.71000,4.590
2,"Nokia Smart Phone, with Caller ID",Technology,169467,3775.20570,3.540
3,"Sharp Wireless Fax, Laser",Technology,55565,3480.74096,2.080
4,"KitchenAid Stove, Silver",Office Supplies,60369,3247.98900,1.240
5,"Hewlett Wireless Fax, Digital",Technology,48978,3057.46320,1.278
6,Hewlett Packard LaserJet 3310 Copier,Technology,47880,2951.95080,1.600
7,"Eldon Lockers, Industrial",Office Supplies,52615,2904.99150,4.540
8,"Cisco Smart Phone, with Caller ID",Technology,77838,2780.05080,3.970
9,"Apple Smart Phone, Cordless",Technology,74042,2771.60700,2.720


In [51]:
# Analyze product categories and subcategories for sales and profits
query = """
SELECT
    Category,
    "Sub.Category",
    SUM(total_sales) AS total_sales,
    SUM(Profit) AS total_profit,
    SUM(Quantity) AS total_quantity
FROM
    superstore
GROUP BY
    Category, "Sub.Category"
ORDER BY
    total_profit DESC
    limit 10;
"""

# Execute the query and fetch results into a DataFrame
with engine.connect() as connection:
    result = connection.execute(text(query))
    df_category_analysis = pd.DataFrame(result.fetchall(), columns=result.keys())

df_category_analysis

,Category,Sub.Category,total_sales,total_profit,total_quantity
0,Technology,Copiers,7174880,258567.54818,7454
1,Technology,Phones,8402406,216717.00580,11870
2,Furniture,Bookcases,7061715,161924.41950,8310
3,Office Supplies,Appliances,5029230,141680.58940,6078
4,Furniture,Chairs,7446742,140396.26750,12336
5,Technology,Accessories,3805586,129626.30620,10946
6,Office Supplies,Storage,5486442,108461.48980,16917
7,Office Supplies,Binders,2289277,72449.84600,21429
8,Office Supplies,Paper,1239813,59207.68270,12822
9,Technology,Machines,3665068,58867.87300,4906


In [52]:
# Assess sales and profit performance by country and city
query = """
SELECT
    Country,
    City,
    SUM(Profit) AS total_profit,
    SUM(total_sales) AS total_sales
FROM
    superstore
GROUP BY
    Country,
    City
ORDER BY
    total_profit DESC
LIMIT 10;
"""

# Execute the query and fetch results into a DataFrame
with engine.connect() as connection:
    result = connection.execute(text(query))
    df_performance_analysis = pd.DataFrame(result.fetchall(), columns=result.keys())

df_performance_analysis


,Country,City,total_profit,total_sales
0,United States,New York City,62036.98370,1263576
1,United States,Los Angeles,30440.75790,873237
2,United States,Seattle,29156.09670,597644
3,Nicaragua,Managua,17853.71804,399001
4,United States,San Francisco,17507.38540,541500
5,Australia,Sydney,16002.80700,609686
6,Austria,Vienna,15660.84000,286052
7,United Kingdom,London,15604.81500,416394
8,El Salvador,San Salvador,15036.51040,287745
9,Mexico,Mexico City,13342.29176,466319


In [53]:
# Total sales, profit, and quantity sold by Product, Country, and Year
query_product = """
SELECT
    "Product.Name",
    Country,
    Year,
    SUM(total_sales) AS total_sales,
    SUM(Profit) AS total_profit,
    SUM(Quantity) AS total_quantity
FROM
    superstore
GROUP BY
    "Product.Name",
    Country,
    Year
ORDER BY
    total_profit DESC
LIMIT 10;
"""

# Execute the query and fetch results into a DataFrame
with engine.connect() as connection:
    result_product = connection.execute(text(query_product))
    df_product_analysis = pd.DataFrame(result_product.fetchall(), columns=result_product.keys())

print("Total Sales, Profit, and Quantity by Product, Country, and Year:")
df_product_analysis

Total Sales, Profit, and Quantity by Product, Country, and Year:


,Product.Name,Country,Year,total_sales,total_profit,total_quantity
0,Canon imageCLASS 2200 Advanced Copier,United States,2014,132300,15679.9552,11
1,Canon imageCLASS 2200 Advanced Copier,United States,2013,121100,9519.9728,9
2,Fellowes PB500 Electric Punch Plastic Comb Bin...,United States,2012,57448,7498.8410,13
3,Ibico EPK-21 Electric Binding System,United States,2011,47250,4630.4755,5
4,"Hoover Stove, Red",Italy,2014,111426,3979.0800,14
5,Hewlett Packard LaserJet 3310 Copier,United States,2014,46200,3623.9396,18
6,Zebra ZM400 Thermal Label Printer,United States,2012,23220,3343.5360,6
7,GBC Ibimaster 500 Manual ProClick Binding System,United States,2013,143448,2967.8220,26
8,"Samsung Smart Phone, VoIP",Japan,2011,76989,2939.3100,11
9,"Apple Smart Phone, with Caller ID",India,2013,51768,2817.9900,9


In [54]:
#  Total sales, profit, and quantity sold by Segment
query_segment = """
SELECT
    segment,
    SUM(total_sales) AS total_sales,
    SUM(profit) AS total_profit,
    SUM(quantity) AS total_quantity
FROM
    superstore
GROUP BY
    segment
ORDER BY
    total_profit DESC;
"""

# Execute the query and fetch results into a DataFrame
with engine.connect() as connection:
    result_segment = connection.execute(text(query_segment))
    df_segment_analysis = pd.DataFrame(result_segment.fetchall(), columns=result_segment.keys())

print("Total Sales, Profit, and Quantity by Segment:")
df_segment_analysis

Total Sales, Profit, and Quantity by Segment:


,segment,total_sales,total_profit,total_quantity
0,Consumer,31654159,749239.78206,92157
1,Corporate,18811317,441208.32866,53565
2,Home Office,11357424,277009.18056,32590


In [55]:
# Total sales, profit, and quantity sold by Category
query_category = """
SELECT
    category,
    SUM(total_sales) AS total_sales,
    SUM(profit) AS total_profit,
    SUM(quantity) AS total_quantity
FROM
    superstore
GROUP BY
    category
ORDER BY
    total_profit DESC
LIMIT 10;
"""

# Execute the query and fetch results into a DataFrame
with engine.connect() as connection:
    result_category = connection.execute(text(query_category))
    df_category_analysis = pd.DataFrame(result_category.fetchall(), columns=result_category.keys())

print("Total Sales, Profit, and Quantity by Category:")
df_category_analysis

Total Sales, Profit, and Quantity by Category:


,Category,total_sales,total_profit,total_quantity
0,Technology,23047940,663778.73318,35176
1,Office Supplies,18686322,518473.83430,108182
2,Furniture,20088638,285204.72380,34954
